# Import librarries

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings('ignore')

# Super vised ML 

# Metrics
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score

# Regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Hyper parameter tuning
from sklearn.model_selection import GridSearchCV

# Loading train, test
# Filtering, Scaling, encoding test data

In [4]:
X_train = pd.read_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/16_X_train_scaled_encoded.csv")
y_train = pd.read_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/13_y_train.csv")
X_test = pd.read_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/14_X_test.csv")
y_test = pd.read_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/15_y_test.csv")
X_test_filtered = X_test.loc[:, X_train.columns]
X_test_filtered.to_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/17_X_test_filtered.csv")
scaler_encoder = joblib.load(r"C:/ML Projects/Diamond Price Prediction/Artifacts/Models/Transform models/feature_scaler_encoder.pkl")
X_test_scaled_encoded = scaler_encoder.fit_transform(X_test_filtered)
X_test_scaled_encoded = pd.DataFrame(X_test_scaled_encoded)
X_test_scaled_encoded.to_csv(r"C:/ML Projects/Diamond Price Prediction/Artifacts/EDA intermediate datasets/18_X_test_scaled_encoded.csv")
X_test = X_test_scaled_encoded

# Models

In [8]:
models ={
            "Linear Regression": LinearRegression(),
        }

# Params

In [5]:
params = {
            "Linear Regression":{}
         }

# Model training and testing

In [9]:
def evaluate_model_regression(X_test, true, predicted) :
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    adj_r2_square = 1 - (1 - r2_square) * (len(true) - 1)/(len(true) - X_test.shape[1] - 1)
    return mse, mae, rmse, r2_square, adj_r2_square 
    
report = {}
model_params = {}
for i in range(len(list(models))):
    model = list(models.values())[i]
    para = params[list(models.keys())[i]]

    gs = GridSearchCV(model, para, cv = 3, scoring = "r2")
    gs.fit(X_train, y_train)

    model.set_params(**gs.best_params_)
    model.fit(X_train, y_train)

    y_test_pred = model.predict(X_test)
    
    values = evaluate_model_regression(X_test, y_test, y_test_pred)
    report[list(models.keys())[i]] = {var: val for var, val in zip(('mse', 'mae', 'rmse', 'r2_square', 'adj_r2_square'), values)}
    model_params[list(models.keys())[i]] = dict(gs.best_params_)

In [10]:
    
print(report)

{'Linear Regression': {'mse': 1209259.6799586136, 'mae': 810.6408367324608, 'rmse': 1099.6634394025355, 'r2_square': 0.9251676095321284, 'adj_r2_square': 0.9251579435125377}}


In [11]:
print(model_params)

{'Linear Regression': {}}


# Best Model

In [12]:
best_model_score = 0
for key1, value1 in report.items() :
    if value1['adj_r2_square'] > best_model_score :
        best_model_score = max(best_model_score, value1['adj_r2_square'])
        best_model = key1
print(best_model, best_model_score)

Linear Regression 0.9251579435125377
